In [2]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
from pyprojroot import here

## Format policies flagged in the "Other climate policy" category

In [3]:
all_policies = ['Paris Agreement', 'Green New Deal', 'Executive action', 'Emergency declaration',
                  '(De)regulation and laws', 'Renewable energy', 'Emission reduction',
                  'Oil and gas industry', 'Other climate policy']

no_other_policies = ['Paris Agreement', 'Green New Deal', 'Executive action', 'Emergency declaration',
                  '(De)regulation and laws', 'Renewable energy', 'Emission reduction',
                  'Oil and gas industry']

only_concrete_policies= ['Paris Agreement', 'Green New Deal', 'Emergency declaration',
                  'Renewable energy', 'Emission reduction',
                  'Oil and gas industry', 'Other climate policy']

regulation_col = '(De)regulation and laws'
executive_action_col = 'Executive action'


In [5]:


# Read the annotation JSON file into a DataFrame
complete_df = pd.read_json(here("data/individual_policy_data/policy-recoding-2025-05-21-complete.json"))
df_with_inner_id = pd.read_json(here("data/individual_policy_data/policy-at-2025-04-27.json"))
# Extract internal_id from the nested 'data' field
df_with_inner_id['internal_id'] = df_with_inner_id['data'].apply(lambda x: x['internal_id'])

# Make a mapping from internal_id -> inner_id
internal_to_inner = dict(zip(df_with_inner_id['internal_id'], df_with_inner_id['inner_id']))


In [6]:
# Filter out NaN values in 'other-explained'
non_nan_other_explained_df = complete_df.loc[complete_df['other_explained'].notna(), ['internal_id', 'annotator', 'other_explained']]
non_nan_other_explained_df['inner_id'] = non_nan_other_explained_df['internal_id'].map(internal_to_inner)


cols = ['inner_id','internal_id', 'other_explained']
non_nan_other_explained_df = non_nan_other_explained_df[cols]
non_nan_other_explained_df.to_csv('output/other_policies_mentioned.csv')


In [7]:
other_df = pd.read_csv(here('data/individual_policy_data/other_climate_policy_summary.csv'))

In [8]:
other_df.drop(columns=['Notes'], inplace=True)

In [9]:
other_df.sort_values(by='Number of transcripts', ascending=False, inplace=True)

In [10]:
other_df[other_df['Policy?']==True][['Specifics','Number of transcripts']].to_csv('output/other_climate_policy_SI_table.csv',index=False)

In [11]:


# Replace specifics with Sup-category when Sup-category is not NaN
other_df['Specifics'] = np.where(other_df['Sup-category'].notna(), other_df['Sup-category'], other_df['Specifics'])


In [12]:
other_df = other_df.drop(columns='Sup-category')

In [13]:
other_df.rename(columns={'Specifics': 'Policy annotation', 'Policy?': 'Policy or not'}, inplace=True)

In [14]:
other_df.to_csv('output/other_climate_policy_clean.csv', index=False)